
*Technical University of Munich<br>
Professorship of Environmental Sensing and Modeling<br><br>*
**Author:**  Daniel Kühbacher<br>
**Date:**  16.01.2024

--- 

# Calculate Cold Start Excess Emissions (CSEE)

<!--Notebook description and usage information-->
This notebook is used to calculate cold start excess emissons using HBEFA emission factors. <br>
Cold start excess emissions refer to the increased release of pollutants that occur when an engine is started from a cold state, typically when it hasn't been running for several hours. During this phase, the engine and exhaust system are not yet at optimal operating temperatures, which impairs combustion efficiency and the effectiveness of emission control devices like catalytic converters, leading to higher levels of pollutants such as carbon monoxide, hydrocarbons, and nitrogen oxides. These emissions are significantly reduced once the engine warms up.<br>

HBEFA provides emission factors for Personal Cars (PC) and Light Cargo Vehicles (LCV) in the unit *"gramm/start"*.<br>
The following parameters can be set:
- Ambient temperature 
- Trip length 
- Parking hours (to determine how hot the engine is before the starting process)

Since city-specific information on trip lenght or parking hours is generally not available, average values for these parameters are provided as well. Temperature information can be retrieved from local weather stations.

## Required input
- Total number daily vehicle starts in the area of interest. This information can be found in traffic models as it constitutes a major input parameter for traffic modeling.
- Hourly ambient temperature for the region of interest.
- Temporal activity profile for extrapolation of the dialy vehicle starts to the whole year

## Output
Total vehicle cold start excess emissions for the area and timeframe of interest. 



In [1]:
# import libraries
import sys
import os
os.environ['USE_PYGEOS'] = '0'

import pandas as pd
import geopandas as gpd

# import custom modules
sys.path.append('../utils')
import data_paths
import traffic_counts
from hbefa_cold_emissions import HbefaColdEmissions

from lmu_meteo_api import interface
from datetime import datetime

# Reload local modules on changes
%reload_ext autoreload
%autoreload 2

# Notebook Settings

In [3]:
# emission components to be calculated
components = ['NOx','PM', 'CO2(rep)', 'CO2(total)',
              'NO2', 'CH4', 'NMHC', 'PM2.5',
              'BC (exhaust)','CO'] 

# Define start and end time for emission calculation
start_date = datetime(2022, 1, 1)
end_date = datetime(2022, 12, 31)

# define filename of the visum file
visum_filename = "visum_links.GPKG"

# if True, the script will only calculate the emission for the area within the roi polygon
clip_to_area = True
roi_polygon = data_paths.MUNICH_BOARDERS_FILE # defines ROI for clipping

# defines the scaling road type for temporal extrapolation
reference_scaling_road_class = 'Distributor/Secondary'

###
#
# STORE RESULTS
#
###

store_result = False
store_path = data_paths.INVENTORY_PATH
def store_filename(year:str):
    return f'linesource_all_munich_{year}_cold.gpkg'

# store temporal profiles
store_temporal_profiles = False
def store_temporal_filename(year:str, vehicle_class:str):
    return f'{year}_{vehicle_class}_temporal_profiles_cold_start.csv'

## Import data

In [4]:
# import visum O-D matricies
visum_links = gpd.read_file(data_paths.VISUM_FOLDER_PATH + visum_filename,
                            driver = 'GPKG')

# calculate starts per squaremeter before gridding
visum_links['PC_starts_per_meter'] = visum_links['PC_cold_starts'] / visum_links['geometry'].length
visum_links['LCV_starts_per_meter'] = visum_links['LCV_cold_starts'] / visum_links['geometry'].length

if clip_to_area:
    roi = gpd.read_file(roi_polygon).to_crs(visum_links.crs)
    visum_links = gpd.clip(visum_links, roi)
    visum_links = visum_links.explode(ignore_index=True) # convert multipolygons to polygons

## Notebook functions

In [5]:
# function to generate annual temperature profile from lmu meteo data in Munich
def annual_temperature_profile(start_date:datetime, 
                               end_date:datetime,
                               aggregate = 'H') -> pd.Series:
    """Downloads meteo data from the LMU Meteo station and returns a dataframe 
    with hourly temperatures for Munich

    Args:
        year (int): Year
        aggregate (str, optional): aggregate to specified timeframe. Defaults to 'H'.

    Returns:
        pd.Series: temperature profile
    """
    
    start_time = start_date.strftime('%Y-%m-%d') + 'T00-00-00'
    end_time = end_date.strftime('%Y-%m-%d') + 'T23-59-59'
    
    if datetime.strptime(end_time, '%Y-%m-%dT%H-%M-%S').date() > datetime.now().date():
        end_time = datetime.now().strftime('%Y-%m-%dT%H-%M-%S')
    
    lmu_api = interface.meteo_data()
    data = lmu_api.get_meteo_data(parameters = ["air_temperature_2m"], 
                                station_id = 'MIM01', 
                                start_time = start_time, 
                                end_time = end_time)
    
    return (data.air_temperature_2m - 273.15).resample(aggregate).mean()

## Initialize objects and download temperature data

In [6]:
# import trafic data, download temperature data and instatiate cold start emission object

# instanciate traffic count object
cycles = traffic_counts.TrafficCounts()

# download temperature data 
temperature = annual_temperature_profile(start_date=start_date,
                                         end_date=end_date) 

# instanciate cold start emission object
cs_obj = HbefaColdEmissions(components=components)

Loaded emission factors from /Users/daniel_tum/Documents/code/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_ColdStart_hbefa.txt


## Calculate total emissions for Munich

In [7]:
# caclulate daily total cold start emissions based on ambient temperature

# prepare parameters for emission calculation
parameters = pd.DataFrame(index = pd.date_range(start = start_date,
                                                end = end_date,
                                                freq='1h'))
parameters['temperature'] = temperature
parameters['hour_factor_PC'] = cycles.timeprofile[reference_scaling_road_class]['PC']
parameters['hour_factor_LCV'] = cycles.timeprofile[reference_scaling_road_class]['LCV']

#calculate daily coldstarts in Munich
daily_PC_starts = (visum_links['PC_starts_per_meter'] * visum_links.length).sum()
daily_LCV_starts = (visum_links['LCV_starts_per_meter'] * visum_links.length).sum()

em_list_pc = list()
em_list_lcv = list()

PC_result = pd.DataFrame()
LCV_result = pd.DataFrame()

for idx, row in parameters.iterrows():
    # get emission factors
    em_PC = cs_obj.calculate_emission_hourly(vehicle_starts = 1,
                                             hourly_temperature=row['temperature'],
                                             vehicle_class='PC',
                                             year = idx.year)
    em_LCV = cs_obj.calculate_emission_hourly(vehicle_starts = 1,
                                              hourly_temperature=row['temperature'],
                                              vehicle_class = 'LCV',
                                              year = idx.year)
    # hourly number of vehicle starts
    hourly_PC_starts = daily_PC_starts * row['hour_factor_PC']
    hourly_LCV_starts = daily_LCV_starts * row['hour_factor_LCV']
    PC_result = pd.concat([PC_result, (em_PC * hourly_PC_starts)], axis=1)
    LCV_result = pd.concat([LCV_result, (em_LCV * hourly_LCV_starts)], axis=1)
    
PC_result = PC_result.transpose().set_index(pd.date_range(start=start_date,
                                                end = end_date,
                                                freq='1h'))
LCV_result = LCV_result.transpose().set_index(pd.date_range(start=start_date,
                                                end = end_date,
                                                freq='1h'))   
LCV_result['vehicle_class'] = 'LCV'
PC_result['vehicle_class'] = 'PC'

# combine results
cold_start_emissions = pd.concat([PC_result, LCV_result], axis = 0)
cold_start_emissions_aggregated = cold_start_emissions.groupby(['vehicle_class'])\
    .resample('1Y').sum(numeric_only = True)
cold_start_emissions_aggregated = cold_start_emissions_aggregated[components]

## Distribute emissions on VISUM model

In [8]:
# distribute cold start emissions on road links

visum_cold_start = visum_links[['PC_cold_starts', 'LCV_cold_starts', 'geometry']].copy()
visum_cold_start['PC_cold_starts_norm'] = visum_cold_start['PC_cold_starts'].divide(visum_cold_start['PC_cold_starts'].sum())
visum_cold_start['LCV_cold_starts_norm'] = visum_cold_start['LCV_cold_starts'].divide(visum_cold_start['LCV_cold_starts'].sum())

visum_cold_start_dict = dict()


for year in [str(year) for year in range(start_date.year, end_date.year + 1)]: 
    for c in components:
        visum_cold_start[f'PC_{c}'] = visum_cold_start['PC_cold_starts_norm']\
            .mul(cold_start_emissions_aggregated.loc['PC', year].iloc[0][c])
        visum_cold_start[f'LCV_{c}'] = visum_cold_start['LCV_cold_starts_norm']\
            .mul(cold_start_emissions_aggregated.loc['LCV', year].iloc[0][c])
            
        visum_cold_start_dict.update({year: visum_cold_start.copy()})


## Store spatial results

In [9]:
# only if store_result is True

if store_result:
    for year, emissions in visum_cold_start_dict.items():
        visum_cold_start_save = emissions.drop(['PC_cold_starts', 'LCV_cold_starts',
                                                'LCV_cold_starts_norm', 'PC_cold_starts_norm'], axis = 1)
        # divide by length to get emissions per km
        _col = visum_cold_start_save.drop('geometry', axis = 1).columns
        visum_cold_start_save[_col] = visum_cold_start_save[_col].divide(visum_cold_start_save.geometry.length*1e-3, axis = 0)

        visum_cold_start_save.to_file(store_path + store_filename(year), driver='GPKG')

## Store temporal profiles

In [10]:
# only if store_temporal_profiles is True

temporal_profile_store_path = store_path +'/temporal_profiles/'

if store_temporal_profiles:
    for year in [str(year) for year in range(start_date.year, end_date.year + 1)]:
        tp_normalized_PC = (PC_result.loc[str(year)]/ PC_result.loc[str(year)].mean())[components]
        tp_normalized_LCV = (LCV_result.loc[str(year)]/ LCV_result.loc[str(year)].mean())[components]
        tp_normalized_PC.to_csv(temporal_profile_store_path + store_temporal_filename(year, 'PC'))
        tp_normalized_LCV.to_csv(temporal_profile_store_path + store_temporal_filename(year, 'LCV'))